In [ ]:
import os
from mistralai import Mistral

api_key = "..."
model = "open-mistral-nemo"

client = Mistral(api_key=api_key)

chat_response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
            "content": "Classify the following tweet to determine if it is's sentiment is positive or negative. Only respond with the exact words 'positive' or 'negative' \n" +
            "Just been for a bike ride on my new bike all the way to Walton gardens and back in 45 mins",
        },
    ]
)
print(chat_response.choices[0].message.content)

positive
